In [1]:
#import the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

 #### Loading data

In [2]:
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns="y")
# print a few data samples
print(data.head())

         x1   x2     x3   x4     x5     x6     x7      x8    x9    x10   x11  \
0   0.06724  0.0   3.24  0.0  0.460  6.333   17.2  5.2146   4.0  430.0  16.9   
1   9.23230  0.0  18.10  0.0  0.631  6.216  100.0  1.1691  24.0  666.0  20.2   
2   0.11425  0.0  13.89  1.0  0.550  6.373   92.4  3.3633   5.0  276.0  16.4   
3  24.80170  0.0  18.10  0.0  0.693  5.349   96.0  1.7028  24.0  666.0  20.2   
4   0.05646  0.0  12.83  0.0  0.437  6.232   53.7  5.0141   5.0  398.0  18.7   

      x12    x13  
0  375.21   7.34  
1  366.15   9.53  
2  393.74  10.50  
3  396.90  19.77  
4  386.40  12.34  


#### Calculating the average RMSE

In [3]:
def calculate_RMSE(w, X, y):
    """This function takes test data points (X and y), and computes the empirical RMSE of 
    predicting y from X using a linear model with weights w. 

    Parameters
    ----------
    w: array of floats: dim = (13,), optimal parameters of ridge regression 
    X: matrix of floats, dim = (15,13), inputs with 13 features
    y: array of floats, dim = (15,), input labels

    Returns
    ----------
    RMSE: float: dim = 1, RMSE value
    """
    RMSE = np.sqrt(1 / y.shape[0] * np.sum((y - X @ w)**2))
    assert np.isscalar(RMSE)
    return RMSE

#### Fitting the regressor

In [4]:
def fit(X, y, lam):
    """
    This function receives training data points, then fits the ridge regression on this data
    with regularization hyperparameter lambda. The weights w of the fitted ridge regression
    are returned. 

    Parameters
    ----------
    X: matrix of floats, dim = (135,13), inputs with 13 features
    y: array of floats, dim = (135,), input labels)
    lam: float. lambda parameter, used in regularization term

    Returns
    ----------
    w: array of floats: dim = (13,), optimal parameters of ridge regression
    """
   
    w = np.linalg.inv(X.T @ X + lam * np.identity(X.shape[1])) @ X.T @ y
    assert w.shape == (13,)
    return w

#### Performing computation

In [5]:
"""
Main cross-validation loop, implementing 10-fold CV. In every iteration 
(for every train-test split), the RMSE for every lambda is calculated, 
and then averaged over iterations.

Parameters
---------- 
X: matrix of floats, dim = (150, 13), inputs with 13 features
y: array of floats, dim = (150, ), input labels
lambdas: list of floats, len = 5, values of lambda for which ridge regression is fitted and RMSE estimated
n_folds: int, number of folds (pieces in which we split the dataset), parameter K in KFold CV

Compute
----------
avg_RMSE: array of floats: dim = (5,), average RMSE value for every lambda
"""
X = data.to_numpy()

# The function calculating the average RMSE
lambdas = [0.1, 1, 10, 100, 200]
n_folds = 10

kf = KFold(n_splits=n_folds)

RMSE_mat = np.zeros((n_folds, len(lambdas)))

for fold_idx, (train_index, test_index) in enumerate(kf.split(X)):
    for lam_idx, lam in enumerate(lambdas):
        # find the best w based on training split
        w = fit(X[train_index], y[train_index], lam)

        # then calculate the RMSE on the test split
        RMSE_mat[fold_idx][lam_idx] = calculate_RMSE(w, X[test_index], y[test_index])

avg_RMSE = np.mean(RMSE_mat, axis=0) # avg_RMSE: array of floats: dim = (5,), average RMSE value for every lambda
assert avg_RMSE.shape == (5,)

In [6]:
# Save results in the required format
np.savetxt("./results.csv", avg_RMSE, fmt="%.12f")